In [ ]:
# default_exp train

# Train

> This module contains a script to train a model.

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
from plant_pathology.dataset import *
from plant_pathology.evaluate import *

from fastai.vision.all import *
from fastcore.script import *
from fastai.callback.wandb import *
from wwf.vision.timm import *
import timm
import wandb
from typing import *
from sys import exit

## Train a Model on Data Split

In [ ]:
#export
def timm_or_fastai_arch(arch: str) -> (Union[Any, str], Callable[..., Learner]):
    try:  # Check if fastai arch
        model = globals()[arch]
        learner_func = cnn_learner
    except KeyError:  # Must be timm arch
        model = arch
        learner_func = timm_learner
    return model, learner_func

In [ ]:
#export
def train(
    epochs: int, lr: Union[float, str], frz: int=1, pre: int=800, re: int=256,
    bs: int=200, fold: int=4, smooth: bool=False, 
    arch: str='resnet18', dump: bool=False, log: bool=False, mixup: float=0.,
    fp16: bool=False, dls: DataLoaders=None, save: bool=False, pseudo: Path=None,
 ) -> Learner:
    # Prep Data, Opt, Loss, Arch
    if dls is None: dls = get_dls_all_in_1(presize=pre, resize=re, bs=bs, val_fold=fold, pseudo=pseudo)
    if log: wandb.init(project="plant-pathology")
    if smooth: loss_func = LabelSmoothingCrossEntropyFlat()
    else:      loss_func = CrossEntropyLossFlat()
    m, learner_func = timm_or_fastai_arch(arch)
    
    # Add callbacks
    cbs = [SaveModelCallback("roc_auc_score", fname=f"model_val_on_{fold}")] if save or log else []
    if log: cbs.append(WandbCallback())
    if mixup: cbs.append(MixUp(mixup))
        
    # Build learner
    print(f"# train exs: {len(dls.train_ds)}, val exs: {len(dls.valid_ds)}")
    learn = learner_func(dls, m, loss_func=loss_func,
                    metrics=[accuracy, RocAuc()], cbs=cbs)
    if dump: print(learn.model); exit()
    if lr=="find": learn.lr_find(); exit()
    if fp16: learn.to_fp16()
        
    # Train
    learn.freeze()
    learn.fit_one_cycle(frz, lr)
    learn.unfreeze()
    learn.fit_one_cycle(epochs, slice(lr/100, lr/2))  # Explore other divs
    return learn

In [ ]:
learn = train(0, 0.001, bs=256, log=False)

# train exs: 1457, val exs: 364


epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,1.992081,1.045056,0.609890,0.756291,00:24


In [ ]:
learn.final_record

(#4) [1.992080569267273,1.0450559854507446,0.6098901033401489,0.756291125185832]

In [ ]:
learn = train(1, 1e-2, frz=1, pre=32, re=32, bs=1024, save=True)

# train exs: 1457, val exs: 364


epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,2.309724,1.625947,0.315934,0.510136,00:36


Better model found at epoch 0 with roc_auc_score value: 0.5101360367629043.


epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,2.236294,1.602961,0.324176,0.512948,00:33


Better model found at epoch 0 with roc_auc_score value: 0.5129479563620204.


In [ ]:
saved_model_path = Path("./models/model_val_on_4.pth") 
assert saved_model_path.exists()
saved_model_path.unlink()

## Train Using Cross-Validation

In [ ]:
#export
def softmax_RocAuc(logits, labels):
    probs = logits.softmax(-1)
    return RocAuc()(probs, labels)

In [ ]:
preds = torch.randn(5, 5)
labels = tensor([1, 2, 3, 4, 5]).unsqueeze(-1)
preds, labels.shape

(tensor([[-1.8089, -1.0325,  0.0798,  0.7559, -0.1851],
         [-1.1922, -0.9417, -0.5189, -0.2025,  0.8527],
         [-0.6867, -0.6207, -0.0188,  1.2670, -0.0194],
         [ 1.7436,  1.4802,  0.0136, -0.0302,  0.2295],
         [ 2.0069, -1.4194, -2.0926,  1.4072,  1.0432]]),
 torch.Size([5, 1]))

In [ ]:
softmax_RocAuc(preds, labels)

0.45

In [ ]:
#export
@call_parse
def train_cv(
    epochs:   Param("Number of unfrozen epochs", int), 
    lr:       Param("Initial learning rate", float), 
    frz:      Param("Number of frozen epochs", int)=1, 
    pre:      Param("Presize", tuple)=(682, 1024), 
    re:       Param("Resize", int)=256,
    bs:       Param("Batch size", int)=256,  
    smooth:   Param("Label smoothing?", store_true)=False, 
    arch:     Param("Architecture", str)='resnet18', 
    dump:     Param("Print model", store_true)=False, 
    log:      Param("Log w/ W&B", store_true)=False,
    save:     Param("Save model based on RocAuc", store_true)=False,
    mixup:    Param("Mixup (0.4 is good)", float)=0.0,
    tta:      Param("Test-time augmentation", store_true)=False,
    fp16:     Param("Use mixed-precision", store_true)=False,
    eval_dir: Param("Evaluate model, save results in dir", Path)=None,
    val_fold: Param("Only do 1 fold (4 is baseline, 9 for all data)", int)=None,
    pseudo:   Param("Path to pseudo labels to train on", Path)=None,
):
    print(locals())
    scores = []
    for fold in range(5):
        if val_fold is not None: fold = val_fold  # Not doing CV
        print(f"\nTraining on fold {fold}")
        learn = train(epochs, lr, frz=frz, pre=pre, re=re, bs=bs, smooth=smooth, 
                      arch=arch, dump=dump, log=log, fold=fold, mixup=mixup,
                      fp16=fp16, save=save, pseudo=pseudo)
        
        if hasattr(learn, "mixup") and tta: learn.remove_cb(MixUp)  # Bug when doing tta w/Mixup
        
        if tta and val_fold != 9:  # There IS a valid set
            preds, lbls = learn.tta()
            res = [f(preds, lbls) for f in [learn.loss_func, accuracy, softmax_RocAuc]]
        else: res = learn.final_record
        scores.append(res)
        
        # Create submission file for this model
        if eval_dir: print("Evaluating"); evaluate(learn, Path(eval_dir)/f"predictions_fold_{fold}.csv", tta=tta)
        
        # Delete learner to avoid OOM
        del learn
        if val_fold is not None: break
    scores = np.array(scores)
    print(f"Scores: {scores}\n")
    if val_fold is None: print(f"Mean: {scores.mean(0)}")

In [ ]:
scores = np.ones((5, 4))
scores.mean(0)

array([1., 1., 1., 1.])

In [ ]:
train_cv(0, 2e-2, pre=64, re=64, bs=512, fp16=True, val_fold=4, tta=True, mixup=0.4, eval_dir="test")

{'epochs': 0, 'lr': 0.02, 'frz': 1, 'pre': 64, 're': 64, 'bs': 512, 'smooth': False, 'arch': 'resnet18', 'dump': False, 'log': False, 'save': False, 'mixup': 0.4, 'tta': True, 'fp16': True, 'eval_dir': 'test', 'val_fold': 4, 'pseudo': None}

Training on fold 4
# train exs: 1457, val exs: 364


epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,2.438146,2.204265,0.277473,0.476844,00:23


/home/brandon/miniconda3/lib/python3.8/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


Evaluating


/home/brandon/miniconda3/lib/python3.8/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/kaggle/plant-pathology/data/plant-pathology-2020/sample_submission.csv'

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_dataset.ipynb.
Converted 02_evaluate.ipynb.
Converted 03_train.ipynb.
Converted 04_generate_pseudo_labels.ipynb.
Converted 05_self_knowledge_distillation.ipynb.
Converted index.ipynb.
